In [35]:
import editdistance
import operator
import cPickle as pickle
from string import ascii_lowercase
import random

### Model

This model is basically a word frequency based model. For each word length, we compute the maximum occuring characters and predict accordingly

In [8]:
def train():
    count = 0
    f = open("train.txt")
    model = {}
    for line in f:
        count+=1
        word = line.strip().split()[0]
        if len(word) not in model:
            model[len(word)] = {}
        for c in word:
            if c not in model[len(word)]:
                model[len(word)][c] = 1
            else:
                model[len(word)][c] += 1
       
    f = open("model.pkl","wb")
    print count
    pickle.dump(model, f, -1) 
    return model

This model uses index based letter frequency

In [43]:
def train1():
    count = 0
    f = open("train.txt")
    model = {}
    for line in f:
        count+=1
        word = line.strip().split()[0]
        if len(word) not in model:
            model[len(word)] = {}
            for i in range(0, len(word)):
                model[len(word)][i] = {}
        i = 0
        for c in word:
            if c not in model[len(word)][i]:
                model[len(word)][i][c] = 1
            else:
                model[len(word)][i][c] += 1
            i += 1
       
    f = open("model1.pkl","wb")
    print count
    pickle.dump(model, f, -1) 
    return model

In [45]:
train1()
trained_model = pickle.load(open('model1.pkl'))

753064


In [63]:
print trained_model

{3: {0: {'a': 635, 'c': 634, 'b': 604, 'e': 596, 'd': 597, 'g': 561, 'f': 536, 'i': 591, 'h': 568, 'k': 526, 'j': 524, 'm': 630, 'l': 558, 'o': 539, 'n': 607, 'q': 280, 'p': 610, 's': 623, 'r': 577, 'u': 500, 't': 609, 'w': 524, 'v': 486, 'y': 320, 'x': 283, 'z': 287}, 1: {'a': 641, 'c': 613, 'b': 593, 'e': 620, 'd': 590, 'g': 539, 'f': 579, 'i': 612, 'h': 547, 'k': 471, 'j': 405, 'm': 617, 'l': 585, 'o': 607, 'n': 575, 'q': 221, 'p': 605, 's': 619, 'r': 605, 'u': 550, 't': 617, 'w': 516, 'v': 510, 'y': 397, 'x': 315, 'z': 256}, 2: {'a': 620, 'c': 620, 'b': 571, 'e': 597, 'd': 584, 'g': 551, 'f': 570, 'i': 613, 'h': 520, 'k': 500, 'j': 359, 'm': 608, 'l': 600, 'o': 556, 'n': 565, 'q': 258, 'p': 605, 's': 636, 'r': 599, 'u': 518, 't': 593, 'w': 480, 'v': 487, 'y': 375, 'x': 465, 'z': 355}}, 4: {0: {'\xc3': 55, '\xc5': 17, '\xc4': 13, '\xce': 4, '\xd1': 3, '\xd0': 2, 'a': 4110, 'c': 3772, 'b': 2641, 'e': 2374, 'd': 2383, 'g': 2231, 'f': 1918, 'i': 2710, 'h': 2050, 'k': 2586, 'j': 1186, '

In [38]:
 # return some letter not in guesses
def getRandomLetter(guesses):
    letters = []
    for c in ascii_lowercase:
        letters.append(c)
    random.shuffle(letters)
    for l in letters:
        if l not in guesses:
            return l

In [46]:
'''
def model(maskedWord, guesses):
    # create model here
    size = len(maskedWord)
    if size not in trained_model:
        return getRandomLetter(guesses)
    for k,v in sorted(trained_model[size].items(), key=operator.itemgetter(1), reverse=True):
        if k not in guesses:
            return k
            break
    return getRandomLetter(guesses)
'''

'\ndef model(maskedWord, guesses):\n    # create model here\n    size = len(maskedWord)\n    if size not in trained_model:\n        return getRandomLetter(guesses)\n    for k,v in sorted(trained_model[size].items(), key=operator.itemgetter(1), reverse=True):\n        if k not in guesses:\n            return k\n            break\n    return getRandomLetter(guesses)\n'

In [70]:
def model(maskedWord, guesses):
    # create model here
    size = len(maskedWord)
    if size not in trained_model:
        return getRandomLetter(guesses)
    for i in range(0, len(maskedWord)):
        if maskedWord[i] == '_':
            for k,v in sorted(trained_model[size][i].items(), key=operator.itemgetter(1), reverse=True):
                if k not in guesses:
                    return k
                    break
    return getRandomLetter(guesses)

### Hangman runner

In [61]:
def hangman(solution):
    guesses = {}
    wrongGuess = 0
    maskedWord = "_"*len(solution)
    while wrongGuess < 8 and '_' in maskedWord:
        predictedCharacter = model(maskedWord, guesses)
        if predictedCharacter in guesses:
            continue
        guesses[predictedCharacter] = 1
        if predictedCharacter in solution:
            chars = list(maskedWord)
            for i in range(0, len(solution)):
                if solution[i] == predictedCharacter:
                    chars[i] = predictedCharacter
            maskedWord = ''.join(chars)
        else:
            wrongGuess += 1

    return maskedWord, guesses

In [41]:
def loss(solution, prediction):
    # find the levenshtein distance between the two words
    return editdistance.eval(solution, prediction)

In [71]:
f = open("test.txt")
count = 0.0
distance = 0.0
for line in f:
    line = line.strip().split(',')[1]
    count += 1
    if count == 1:
        continue
    predictedWord, guesses = hangman(line)
    distance += loss(line, predictedWord)
print distance/count

2.42090116018
